In [3]:
#pip install gensim nltk spacy

In [39]:
import spacy
import es_core_news_sm
import gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import unidecode
import re

#for bigrams
from gensim.models import Phrases


In [25]:
# ! python -m spacy download es_core_news_sm

### Reading the data

In [31]:
data = pd.read_csv('C:/Users/carme/Dropbox/MIO/DOCS/Hertie/Thesis/Hertie-Thesis/Data/clean_data.csv')
# 'C:/Users/carme/Dropbox/MIO/DOCS/Hertie/Thesis/Elecciones_Municipales_2024/clean_data.csv'
#filter column with na values
plans = data.copy().dropna()
plans = plans.reset_index(drop=True)
#plans.shape

# Encontrar duplicados en 'clean_text'
duplicated_texts = plans['clean_text'][plans['clean_text'].duplicated(keep=False)]

# Mostrar los documentos duplicados
# print(duplicated_texts)

# plans.loc[303,'clean_text']
plans.at[303,'clean_text'] = float("NaN")
# plans.loc[303,'clean_text']
plans = plans.dropna()
# Ensure all entries in 'clean_text' are strings and handle missing values
data['clean_text'] = data['clean_text'].fillna('').astype(str)
plans.shape

(309, 10)

### Preprocessing

In [32]:
#SIN BIGRAMS y sin stop words
# # Load the Spanish NLP model
# nlp = spacy.load('es_core_news_sm')

# # Preprocess documents: tokenize, remove stopwords and punctuation, and lemmatize
# def preprocess(doc):
#     spacy_doc = nlp(doc.lower())
#     return [token.lemma_ for token in spacy_doc if not token.is_stop and not token.is_punct]

# processed_docs = data['clean_text'].apply(preprocess)



In [47]:
# CON BIGRAMS
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))  # Using Spanish stopwords

# Tokenize documents
def preprocess(text):
    # Convert to lower case and remove accents
    text = text.lower()
    # Tokenize and remove stopwords
    tokens = [word for word in text.split() if word not in stop_words]
    tokens = [unidecode.unidecode(word) for word in tokens]
    return tokens

tokenized_docs = [preprocess(doc) for doc in plans['clean_text']]

# Build bigrams
bigram = Phrases(tokenized_docs, min_count=1, threshold=2) # Play with these parameters according to your specific needs
bigram_mod = gensim.models.phrases.Phraser(bigram)

# Apply the trained bigram model to each document
docs_with_bigrams = [bigram_mod[doc] for doc in tokenized_docs]

# Create a dictionary representation of the documents
dictionary = Dictionary(docs_with_bigrams)

# Create a corpus from the dictionary and tokenized documents
corpus = [dictionary.doc2bow(doc) for doc in docs_with_bigrams]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Predefined topics

In [48]:
# Define seed words for topics, should be tops bigrams
seed_words = {
    'Seguridad': ['seguridad', 'homicidio', 'asaltos','policia','hurto','hurtos','robo','robos','tacha'],
    'Capital Social': ['abstencionismo', 'participacion elecciones'],
    'Transformacion Digital': ['digital','gobernanza datos','municipalidad electronica', 'e municipalidad', ], #muni electronica y e muni son de los tramites, pero son digital, segun yo no deberian ir aca?
    'Tramites Simples': ['plan de mejora regulatoria','permiso de construccion','permisos de construccion','uso de suelos'],
    'Transparencia Municipal': ['transparencia','denuncia','acceso informacion','rendicion cuentas','participacion ciudadana','datos abiertos'],
    'Administracion Presupuestaria': ['presupuesto','ejecucion recursos','morosidad','dependencia financiera'],
    #'Compromiso con la sostenibilidad': ['residuos recolectados','basura','residuos','reciclaje','valoracion residuos','medio ambiente','sostenibilidad'], #estas dos ultimas creo que deberia estar en otra categoria y deberia ser solo de recoleccion de residuos
    #lo voy a dividir en dos:
    'Recoleccion de Residuos':['residuos','reciclaje'],
    'Medio Ambiente':['medio ambiente','sostenibilidad'],
    'Infraestructura de Transporte': ['red vial','vias','obras','infraestructura transporte','km','kilometro'],
    'Conectividad Vial': ['conectividad vial','traslado','earopuerto','puerto','presas','atraso carretera'],
    'Acceso a Servicios Publicos': ['agua','potable','tuberia','acueducto','electricidad','eliminacion de excremento'], #no inclui acceso a camion recolector de basura
    'Servicios Publicos Municipales': ['inversion servicios','servicios municipales','servicios publicos','aseo vias','aseo sitios'], #no estoy incluyendo inversion en cultura, deporte y educacion
    'Redes de Datos (internet)': ['datos moviles','velocidad descarga'], #no inclui 3g y 4g 
    'Redes de Voz Movil (telefonia)': ['telefonia','telefonica','llamadas telefonicas'], #no inclui 2g y 3g
    'Redes de Datos Fijas (internet)': ['banda ancha','operadoras internet'],
    'Acceso a TICs en Hogares': ['acceso computadoras','acceso internet'],
    'Acceso a TICs en el Sistema Educativo': ['estudiantes computadora','pronie'],
    'Salud': ['esperanza vida','mortalidad infantil','fertilidad adolescentes'],
    'Escolaridad Media de Fuerza Laboral Actual': ['escolaridad','secundaria concluida'],
    'Habilidades Tecnicas de Fuerza Laboral Actual': ['tecnicos graduados'],
    #habilidades y competencias podria ser uno solo
    'Habilidades en Ciencia y Tecnologia de Fuerza Laboral Actual': ['ciencia tecnologia','ingenieria tecnologia','ciencias medicas','ciencias exactas','ciencias naturales','ciencias agricolas'],
    'Cobertura Educativa': ['alumnos','educacion primaria','educacion secundaria'],
    'Curriculo Completo': ['ingles','curriculo completo'], #deje afuera informatica
    'Competencias Basicas': ['matematica','espanol','evaluacion estandarizada','segundo idioma'], #deje afuera cientifica y secundaria
    'Encadenamientos Productivos': ['encadenamientos productivos','relacion venta','relacion compra'],
    'Produccion': ['pib capita','pib per capita','actividades economicas','actividad economica','emprendimiento'], #agregue emprendimiento
    'Exportacion de Bienes': ['exportaciones','productos primarios'],
    'Sector Constructivo': ['construcciones','construccion'],
    'Sector Electrico': ['interrupciones electricas','electricidad','electrico'],
    'Sector Laboral': ['desempleo','empleo','asalariado','asalariada','asegurado independiente'],
    'Genero':['brecha','genero','mujeres','mujer','desigualdad'], #revisar desigualdad
    'Cultura': ['cultura','recreacion','arte'],
    'Deporte':['deporte'],
    'Turismo': ['turismo','turistas']
}

In [ ]:
# # Boost seed words by repeating them in the documents (optional step)
# boosted_documents = []
# for doc in processed_docs:
#     boosted_doc = doc[:]
#     for topic, seeds in seed_words.items():
#         for seed in seeds:
#             if seed in doc:
#                 boosted_doc.extend([seed] * 10)  # Repeat seed words to increase their frequency
#     boosted_documents.append(boosted_doc)

# Create a dictionary and corpus
# dictionary = Dictionary(boosted_documents)
# corpus = [dictionary.doc2bow(doc) for doc in boosted_documents]

In [49]:
# Train LDA
lda = LdaModel(corpus=corpus, 
               id2word=dictionary, 
               num_topics=len(seed_words), 
               passes=15, random_state=0)

In [ ]:
# Display topics
topics = lda.print_topics()
for topic_id, topic in topics:
    print(f"Topic {topic_id}: {topic}")

Topic 6: 0.011*"*" + 0.003*"gobierno_2024-2028" + 0.003*"adn" + 0.002*"canton." + 0.002*"proyectos" + 0.002*"canton" + 0.002*"desarrollo" + 0.002*"gestion" + 0.001*"cantonal" + 0.001*"municipalidad"
Topic 17: 0.004*"jesus_alcalde" + 0.003*"-" + 0.003*"*" + 0.002*"canton" + 0.001*"km_-" + 0.001*"coordinacion" + 0.001*"estrategias" + 0.001*"*_apoyo" + 0.001*"-_coordinar" + 0.001*"comunidades"
Topic 19: 0.004*"-" + 0.002*"municipalidad" + 0.002*"ministerio" + 0.002*"objetivo_lineas" + 0.002*"san_mateo." + 0.002*"accion_responsables" + 0.001*"-_coordinar" + 0.001*"plan_gob" + 0.001*"ierno_2024-2028" + 0.001*"mejorar"
Topic 23: 0.003*"canton" + 0.003*"desarrollo" + 0.002*"grecia" + 0.002*"mas" + 0.002*"liberia" + 0.002*"canton." + 0.002*"municipalidad" + 0.002*"gestion" + 0.002*"nicoya" + 0.002*"municipal"
Topic 30: 0.003*"*" + 0.003*"canton" + 0.002*"!rio_cuarto" + 0.002*"*_implementar" + 0.002*"pasj_puertas" + 0.002*"gobierno_-" + 0.002*"alianza_san" + 0.002*"abiertas_plan" + 0.002*"jose_